# Plots of shadowing transform
Refer to: https://www.gaussianwaves.com/2013/09/log-distance-path-loss-or-log-normal-shadowing-model/


In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import DatasetMetadata
from torchsig.signals.builders.tone import ToneSignalBuilder
import torchsig.transforms.functional as F
from torchsig.utils.dsp import (
    torchsig_complex_data_type
)

import numpy as np
from scipy import stats

%matplotlib inline
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

In [ ]:
def generate_tone_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband tone Signal.

    Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 128.
        scale (int, optional): Scale factor for normalized signal data. Defaults to 1.0.

    Returns:
        signal: generated Signal.

    """
    rng = np.random.default_rng(42)
    sample_rate = 10e6
    
    md = DatasetMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_max = 1,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 10.0,
        snr_db_max = 10.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0, 
        transforms = [],
        target_transforms = [],
        class_list = ['tone'],
        class_distribution = [1.0],
        num_samples = 1,
        seed = 1234
    )
    builder = ToneSignalBuilder(
        dataset_metadata = md, 
        class_name = 'tone',
        seed = 1234
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale).astype(torchsig_complex_data_type)

    return signal

In [ ]:
rng = np.random.default_rng(42)

N = 10000
sampling_rate = 4.0
tone_bb_data = generate_tone_signal(num_iq_samples = N, scale = 1.0).data 
tone_data = tone_bb_data * np.exp(2j * np.pi * 0.2 * np.arange(N) / sampling_rate)
tone_power = np.mean(np.abs(tone_data) ** 2)

# point check
mean_db = 0
sigma_db = 3
power_db = rng.normal(mean_db, sigma_db) # normal distribution in log domain
#power_db = -3.0

new_tone = tone_data * 10 ** (power_db / 20)
new_power = np.mean(np.abs(new_tone) ** 2)

print("power_db: ",power_db)
print(tone_power)
print(new_power)
print("ratio: ",10*np.log10(new_power) - 10*np.log10(tone_power))

In [ ]:
rng = np.random.default_rng(42)

# functional check
n_iterations = 30
results = [
    10*np.log10(np.mean(np.abs(
        F.shadowing(
            data = tone_bb_data,
            mean_db = 4.0,
            sigma_db = 2.0,
            rng = rng
        )**2)
    ))
    for _ in  range(n_iterations)
]
results_array = np.array(results)

# Shapiro-Wilk test for normality
stat, p_value = stats.shapiro(results_array)

print(f"Shapiro-Wilk test statistic: {stat:.3f}, p-value: {p_value:.3f}")
if p_value > 0.05:
    print("The distribution is likely normal")
else:
    print("The distribution is likely not normal")

plt.plot(results_array)